In [14]:
#import libraries
import pandas as pd
import os
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from scipy import stats
import scipy 
import numpy as np
from tqdm.notebook import tqdm
from torch import nn, optim
import time  
import torch 
import random
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from datetime import date
import pickle
import sys
from pathlib import Path  


In [48]:
path= r'D:\Study 2 Data\wetransfer_velocity-knees-flexion-extension-csv_2022-04-14_1349'

# Lists the name of all the files available in path
train_files = sorted(os.listdir(path))
print("Current working directory has been changed ...")

# Changes the working directory to get the data from their location 
os.chdir(path)
cwd = os.getcwd()
print(f'Current working directory is: {cwd}')
print(f"There are {len(train_files)} files in the specified path.")


Current working directory has been changed ...
Current working directory is: D:\Study 2 Data\wetransfer_velocity-knees-flexion-extension-csv_2022-04-14_1349
There are 40 files in the specified path.


In [49]:
p = ['n','r','c','l', 'a']
p

['n', 'r', 'c', 'l', 'a']

In [50]:
n = sorted(p)
n

['a', 'c', 'l', 'n', 'r']

In [51]:
# all_data = pd.DataFrame()
# i = 0

# for f in tqdm(train_files):
#     if i == 0: #include time for the first file
#         # print(f[0:-4]) #print file name without csv
#         file_data = pd.read_csv(f)
#         file_data.rename(columns={file_data.columns[1]: f[0:-4] + ' Right', file_data.columns[-1]: f[0:-4] + ' Left' }, inplace=True)
#         all_data = pd.concat([all_data, file_data], axis=1) #Concatenate data 
#         i = 1 

#     else:
#         # print(f)
#         if 'Pelvis' in f or 'Torso' in f: #Pevlis and Torso do not have right and left values, treat them seperatly

#             print(f[0:-4]) #print file name without csv
#             file_data = pd.read_csv(f)
#             file_data.drop(labels='Time', axis=1, inplace=True) #remove time column

#             for c in range(len(file_data.columns)):
#                 file_data.rename(columns={file_data.columns[c]: f[0:-4] + ' - ' + file_data.columns[c]}, inplace=True) #rename colums file title + column name

#             all_data = pd.concat([all_data, file_data], axis=1) #Concatenate data 
            
#         else:
#             print(f[0:-4]) #print file name without csv
#             file_data = pd.read_csv(f)
#             file_data.drop(labels='Time', axis=1, inplace=True) #remove time column
#             # print(len(file_data.columns))
#             if len(file_data.columns) > 2:
#                 print('Error: Number of columns is greater that 2. Make sure they are included.')
#                 break 

#             file_data.rename(columns={file_data.columns[0]: f[0:-4] + ' Right', file_data.columns[1]: f[0:-4] + ' Left' }, inplace=True)
#             all_data = pd.concat([all_data, file_data], axis=1) #Concatenate data 

    

In [52]:
# all_data

In [53]:

# os.makedirs('D:\Study 2 Data\Test', exist_ok=True)  #check if directory exists
# all_data.to_csv('D:\Study 2 Data\Test/all_data.csv')  #export data to csv

In [54]:
# all_data.to_excel("D:\Study 2 Data\Trial Data/all_data.xlsx")  

In [55]:
other_features = ['Acceleration Pelvis', 'Angle to vertical', 'Barometer', 'Battery', 'Local Acceleration', 'Local Gyroscope', 'Local Magnetometer', 'Pelvis Velocity', 'Pelvis', 'Rotation Angle', 'Signal Strength', 'Torso Velocity', 'Torso', 'Velocity Pelvis' ]
sub_features_3 = ['Rotation Angle', 'Local Magnetometer','Local Gyroscope', 'Angle to vertical', 'Local Acceleration'] #features where the sensor name is used and each has x, y, z value
sub_features_1 = ['Barometer', 'Signal Strength', 'Battery'] #features where the sensor name is used and each has one column only

In [56]:
sub_features_3_labels = ['Sacrum X', 'Sacrum Y', 'Sacrum Z',
                        'Left thigh X', 'Left thigh Y', 'Left thigh Z',
                        'Right thigh X', 'Right thigh Y', 'Right thigh Z',
                        'Left leg X', 'Left leg Y', 'Left Leg Z',
                        'Right leg X', 'Right leg Y', 'Right leg Z',
                        'Left foot X', 'Left foot Y', 'Left foot Z',
                        'Right foot X', 'Right foot Y', 'Right foot Z',
                        'Dorsal X', 'Dorsal Y', 'Dorsal Z']


sub_features_1_labels = ['Sacrum',
                        'Left thigh', 
                        'Right thigh', 
                        'Left leg', 
                        'Right leg', 
                        'Left foot', 
                        'Right foot',
                        'Dorsal']

In [57]:
sub_features_3_labels[0][:-2]

'Sacrum'

In [58]:
all_data = pd.DataFrame() #create an empty array
i = 0 #initialise index

for f in tqdm(train_files): 
    
    if f[0:-4] in other_features: #Some features do not have right and left legs, treat them seperatly

        if i == 0: #check if this is the first feature
            print(f[0:-4]) #print file name without csv
            file_data = pd.read_csv(f)

            ############
            #Only go through these loops if feature in sub_feature_1 or sub_feature_3
            if f[0:-4] in sub_features_3:
                 for c in range(len(file_data.columns)):
                
                    if c == 0: #skip the first column which contains the Time variable 
                        continue
                
                    file_data.rename(columns={file_data.columns[c]: f[0:-4] + ' - ' + file_data.columns[c] + ' ' + sub_features_3_labels[c-1][:-2]}, inplace=True) #rename colums file title + column name

            elif f[0:-4] in sub_features_1:
                for c in range(len(file_data.columns)):
                
                    if c == 0: #skip the first column which contains the Time variable 
                        continue
                
                    file_data.rename(columns={file_data.columns[c]: f[0:-4] + ' - ' + file_data.columns[c] + ' ' + sub_features_1_labels[c-1]}, inplace=True)
            ############

            else:
                for c in range(len(file_data.columns)):
                
                    if c == 0: #skip the first column which contains the Time variable 
                        continue
                    
                    file_data.rename(columns={file_data.columns[c]: f[0:-4] + ' - ' + file_data.columns[c]}, inplace=True) #rename colums file title + column name

            all_data = pd.concat([all_data, file_data], axis=1) #Concatenate data 
            i = 1 #Change condition


        else:

            print(f[0:-4]) #print file name without csv
            file_data = pd.read_csv(f)
            file_data.drop(labels='Time', axis=1, inplace=True) #remove time column
            
            ############
            #Only go through these loops if feature in sub_feature_1 or sub_feature_3
            if f[0:-4] in sub_features_3:
                for c in range(len(file_data.columns)):                
                    file_data.rename(columns={file_data.columns[c]: f[0:-4] + ' - ' + file_data.columns[c] + ' ' + sub_features_3_labels[c][:-2]}, inplace=True) #rename colums file title + column name

            elif f[0:-4] in sub_features_1:
                for c in range(len(file_data.columns)):                
                    file_data.rename(columns={file_data.columns[c]: f[0:-4] + ' - ' + file_data.columns[c] + ' ' + sub_features_1_labels[c]}, inplace=True)
            # ############
            else:
                for c in range(len(file_data.columns)):
                    file_data.rename(columns={file_data.columns[c]: f[0:-4] + ' - ' + file_data.columns[c]}, inplace=True) #rename colums file title + column name

            all_data = pd.concat([all_data, file_data], axis=1) #Concatenate data 
        
    else: #Loop for files that contain left and right legs 

        if i == 0:
            print(f[0:-4]) #print file name without csv
            file_data = pd.read_csv(f)

            if len(file_data.columns) > 3: #check if the file contains more than 3 columns (Time + left + right legs)
                print(f'Error: Number of columns is in ({f}) greater that 2. Make sure they are included.')
                break 

            if 'Right' in file_data.columns[1]:
                file_data.rename(columns={file_data.columns[1]: f[0:-4] + ' Right', file_data.columns[-1]: f[0:-4] + ' Left' }, inplace=True)
                all_data = pd.concat([all_data, file_data], axis=1) #Concatenate data 
                i = 1 
            else:
                print(f'Error: First column in ({f}) does not contain values for the right leg.')
                break           

        else:
            print(f[0:-4]) #print file name without csv
            file_data = pd.read_csv(f)
            file_data.drop(labels='Time', axis=1, inplace=True) #remove time column
            # print(len(file_data.columns))
            if len(file_data.columns) > 2: #check if the file contains more than 2 columns
                print(f'Error: Number of columns is in ({f}) is greater that 2. Make sure they are included.')
                break 
            
            if 'Right' in file_data.columns[0]: #check right is in first column
                file_data.rename(columns={file_data.columns[0]: f[0:-4] + ' Right', file_data.columns[1]: f[0:-4] + ' Left' }, inplace=True)
                all_data = pd.concat([all_data, file_data], axis=1) #Concatenate data 
            
            else:
                print(f'Error: First column in ({f}) does not contain values for the right leg.')
                break
    

  0%|          | 0/40 [00:00<?, ?it/s]

Acceleration Ankles Eversion-Inversion
Acceleration Ankles Rotation
Acceleration Hips Abduction-Adduction
Acceleration Hips Flexion-Extension
Acceleration Hips Rotation
Acceleration Knees Abduction-Adduction
Acceleration Knees Flexion-Extension
Acceleration Knees Rotation
Acceleration Pelvis
Angle to vertical
Ankles Dorsiflexion-Plantarflexion
Ankles Eversion-Inversion
Ankles Rotation
Barometer
Battery
Hips Abduction-Adduction
Hips Flexion-Extension
Hips Rotation
Knees Abduction-Adduction
Knees Flexion-Extension
Knees Rotation
Local Acceleration
Local Gyroscope
Local Magnetometer
Pelvis Velocity
Pelvis
Rotation Angle
Signal Strength
Torso Velocity
Torso
Velocity Ankles Dorsiflexion-Plantarflexion
Velocity Ankles Eversion-Inversion
Velocity Ankles Rotation
Velocity Hips Abduction-Adduction
Velocity Hips Flexion-Extension
Velocity Hips Rotation
Velocity Knees Abduction-Adduction
Velocity Knees Flexion-Extension
Velocity Knees Rotation
Velocity Pelvis


In [59]:
for c in all_data.columns:
    print(c)

Time
Acceleration Ankles Eversion-Inversion Right
Acceleration Ankles Eversion-Inversion Left
Acceleration Ankles Rotation Right
Acceleration Ankles Rotation Left
Acceleration Hips Abduction-Adduction Right
Acceleration Hips Abduction-Adduction Left
Acceleration Hips Flexion-Extension Right
Acceleration Hips Flexion-Extension Left
Acceleration Hips Rotation Right
Acceleration Hips Rotation Left
Acceleration Knees Abduction-Adduction Right
Acceleration Knees Abduction-Adduction Left
Acceleration Knees Flexion-Extension Right
Acceleration Knees Flexion-Extension Left
Acceleration Knees Rotation Right
Acceleration Knees Rotation Left
Acceleration Pelvis - Pelvis Tilt
Acceleration Pelvis - Pelvis Obliquity
Acceleration Pelvis - Pelvis Rotation
Angle to vertical - QUIXOTE Axis X Sacrum
Angle to vertical - QUIXOTE Axis Y Sacrum
Angle to vertical - QUIXOTE Axis Z Sacrum
Angle to vertical - HELIO Axis X Left thigh
Angle to vertical - HELIO Axis Y Left thigh
Angle to vertical - HELIO Axis Z Lef

In [60]:

os.makedirs('D:\Study 2 Data\exported-data', exist_ok=True)  #check if directory exists
all_data.to_csv('D:\Study 2 Data\exported-data/exported-data3.csv')  #export data to csv